In [3]:
pip install grewpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import grewpy
from grewpy import Corpus, Request, CorpusDraft
import re
from transformation import dico_transformation, phonemes_bigraphiques, consonnes,liste_c_indesirables 

connected to port: 51030


In [2]:
treebank_path="/Users/maria23paz/Documents/Stage/Depo/SUD_Naija-NSC/not-to-release"
grewpy.set_config("sud") # ud or basic
corpus = Corpus(treebank_path)
print(type(corpus))

<class 'grewpy.corpus.Corpus'>


In [3]:
n_sentencens = len(corpus)
sent_ids = corpus.get_sent_ids()

print(f"{n_sentencens=}")
print(f"{sent_ids[200]=}")

n_sentencens=7627
sent_ids[200]='KAD_17_Turkeys_MG__80'


In [5]:
draft=CorpusDraft(corpus)



In [6]:
'''Extrait toutes les syllabes des fichiers conllu'''
def extraire_syllabes(draft):
    pattern= re.compile(r'^Syl\d+$')
    set_syllabes=set()
    for i in range(len(draft)):
        sentence = draft[i]
        features = sentence.features
        for id, dico in features.items():
            dico_copy = dico.copy()
            for key in dico_copy: 
                    if pattern.match(key):            
                        set_syllabes.add(dico[key])
    liste_syllabes=list(set_syllabes)

    return liste_syllabes
'''Prend la liste complète de syllabes et extrait tous les caractères'''
def extraire_caracteres(liste_syllabes):
    liste_caracteres=[]
    for syllabe in liste_syllabes:
        for car in syllabe:
            liste_caracteres.append(car)      
    return liste_caracteres

'''
Prend la lyste de syllabes, prend chaque syllabe et la divise en trois parties.
Prend les listes "phonemes_bigraphiques" et "consonnes" de transformation.py
'''
def diviser_syllabe(liste_syllabes):
    syllabes_divisees = []
    
    for syllabe in liste_syllabes:
        syllabes_char = []
        i = 0
        while i < len(syllabe):
            trouve_phonemebi = False
            for phoneme in phonemes_bigraphiques:
                if re.match(phoneme, syllabe[i:]):
                    syllabes_char.append(phoneme)
                    i += len(phoneme)
                    trouve_phonemebi = True
                    break
            if not trouve_phonemebi:
                if not trouve_phonemebi:
                    if i < len(syllabe) - 2 and all(caracter in consonnes for caracter in syllabe[i:i+3]):
                        consonant_group = syllabe[i:i+3]
                        syllabes_char.append(consonant_group)
                        i += 3
                    elif i < len(syllabe) - 1 and all(caracter in consonnes for caracter in syllabe[i:i+2]):
                        consonant_group = syllabe[i:i+2]
                        syllabes_char.append(consonant_group)
                        i += 2
                    else:
                        syllabes_char.append(syllabe[i])
                        i += 1
        syllabes_divisees.append(syllabes_char)
    return syllabes_divisees  

'''Montre les occurrences de chaque caractère'''
def comptage(liste_caracteres):
    dico_occurrences={}
    for caractere in liste_caracteres:
        quantite=liste_caracteres.count(caractere)
        dico_occurrences[caractere]=quantite
    liste_occurrences=list(dico_occurrences.items())
    return dico_occurrences

'''Prend la liste complete de syllabes, trouve les caractères bizarres et fait un diccionanite avec les syllabes qui le possédent'''
def trouver(liste_syllabes):
    dico_c_indesirables={}
    for c in liste_c_indesirables:
        dico_c_indesirables[c] = []

    for syllabe in liste_syllabes:
        for car in liste_c_indesirables:
            if car in syllabe:
                dico_c_indesirables[car].append(syllabe)

    return dico_c_indesirables
'''
Trouve un caractère indésirable dans les syllabes et le remplace. On remplace "key" par "valeur"
Prend le diccionnaire  avec des caractères indésirables "dico_transformation" de transformation.py
'''
def transformation(liste_syllabes):
    nouvelle_liste=[]
    for syllabe in liste_syllabes:
        nouvelle_syllabe = syllabe
        for key, value in dico_transformation.items():
            if key in nouvelle_syllabe:
                nouvelle_syllabe = nouvelle_syllabe.replace(key, value)
        nouvelle_liste.append(nouvelle_syllabe)
    return nouvelle_liste


def main():
    import pandas as pd
    draft = CorpusDraft(corpus)
    complete_syllabes=extraire_syllabes(draft) 
    transforme_syllabes=transformation(complete_syllabes)    
    complete_phonemes=diviser_syllabe(transforme_syllabes) 
    #complete_caracteres=extraire_caracteres(complete_syllabes) 
    #occurrences_caracteres=comptage(complete_caracteres)
    # syllabes_bizarres=trouver(complete_syllabes)

   
    df=pd.DataFrame({'Syllabes_originales': complete_syllabes, 'Syllabes_transformees':transforme_syllabes, "Division":complete_phonemes})
    df.to_csv('syllabes_divisees6.csv', index=False)

    
if __name__== "__main__" :
    main()                
            